# Radar Data Streaming and Pre-Processing

### This notebook demonstrates how to read the data from the radar using python wrapper for Infineon Radar SDK

#### Importing required libraries 

    - ifxdaq --> is the python wrapper for Radar SDK
    - processing --> is python module for basic processing to get the range-Dopplers from the signal
    - fft --> contains the range_fft and doppler_fft methods
    

In [ ]:
#pip install ifxdaq-3.0.1-py3-none-win_amd64.whl

In [ ]:
#pip install processing.py

In [1]:
import ifxdaq
import processing
import numpy as np
#print(ifxdaq.__version__)
from ifxdaq.sensor.radar_ifx import RadarIfxAvian
import matplotlib.pyplot as plot
import pandas as pd

#### In the folder 'radar_configs' you find radar configuration parameters. It contains radar parameters :

    - transmit power level
    - start/end frequency
    - sampling rate
    - # of chirps per frame
    - # of samples per chirp
    - # of transmit/receive antennas
    - MIMO mode selection

In [2]:
config_file = "radar_configs/RadarIfxBGT60.json"
number_of_frames = 3000

In [3]:
## Run this to understand the current radar settings better
import json
with open(config_file) as json_file:
    c = json.load(json_file)["device_config"]["fmcw_single_shape"]
    chirp_duration = c["num_samples_per_chirp"]/c['sample_rate_Hz']
    frame_duration = (chirp_duration + c['chirp_repetition_time_s']) * c['num_chirps_per_frame']
    print("With the current configuration, the radar will send out " + str(c['num_chirps_per_frame']) + \
          ' signals with varying frequency ("chirps") between ' + str(c['start_frequency_Hz']/1e9) + " GHz and " + \
          str(c['end_frequency_Hz']/1e9) + " GHz.")
    print('Each chirp will consist of ' + str(c["num_samples_per_chirp"]) + ' ADC measurements of the IF signal ("samples").')
    print('A chirp takes ' + str(chirp_duration*1e6) + ' microseconds and the delay between the chirps is ' + str(c['chirp_repetition_time_s']*1e6) +' microseconds.')
    print('With a total frame duration of ' + str(frame_duration*1e3) + ' milliseconds and a delay of ' + str(c['frame_repetition_time_s']*1e3) + ' milliseconds between the frame we get a frame rate of ' + str(1/(frame_duration + c['frame_repetition_time_s'])) + ' radar frames per second.')

With the current configuration, the radar will send out 64 signals with varying frequency ("chirps") between 60.5 GHz and 61.5 GHz.
Each chirp will consist of 128 ADC measurements of the IF signal ("samples").
A chirp takes 64.0 microseconds and the delay between the chirps is 400.0 microseconds.
With a total frame duration of 29.696 milliseconds and a delay of 50.0 milliseconds between the frame we get a frame rate of 12.547681188516362 radar frames per second.


#### the next code block initiliaze the radar with set configuration and loop through the continuous frames; when it  reaches desired number of frames, we process it to get the range-doppler information out of it.

In [7]:
raw_data    = []

with RadarIfxAvian(config_file) as device:                             # Initialize the radar with configurations
    
    for i_frame, frame in enumerate(device):                           # Loop through the frames coming from the radar
        
        raw_data.append(np.squeeze(frame['radar'].data/(4095.0)))      # Dividing by 4095.0 to scale the data
        if(i_frame == number_of_frames-1):
            data = np.asarray(raw_data)
            range_doppler_map = processing.processing_rangeDopplerData(data)
            #del data
            break      

In [8]:
np.save('ed16_raw_data_c3',raw_data)

In [ ]:
#np.save('data',data)

In [19]:
data00 = np.load('acquisizioni_nave/nave_raw_data_c0e0.npy')
data00 = np.asarray(data00)
rdm00 = processing.processing_rangeDopplerData(data00)

In [20]:
print(data00.shape)
range_doppler_map.shape

(1000, 3, 64, 128)


(1000, 3, 64, 64)

In [21]:
data01 = np.load('acquisizioni_nave/nave_raw_data_c0e1.npy')
data01 = np.asarray(data01)
rdm01 = processing.processing_rangeDopplerData(data01)

In [22]:
data02 = np.load('acquisizioni_nave/nave_raw_data_c0e2.npy')
data02 = np.asarray(data02)
rdm02 = processing.processing_rangeDopplerData(data02)

In [23]:
data10 = np.load('acquisizioni_nave/nave_raw_data_c1e0.npy')
data10 = np.asarray(data10)
rdm10 = processing.processing_rangeDopplerData(data10)

In [24]:
data11 = np.load('acquisizioni_nave/nave_raw_data_c1e1.npy')
data11 = np.asarray(data11)
rdm11 = processing.processing_rangeDopplerData(data11)

In [25]:
data12 = np.load('acquisizioni_nave/nave_raw_data_c1e2.npy')
data12 = np.asarray(data12)
rdm12 = processing.processing_rangeDopplerData(data12)

In [26]:
data2 = np.load('acquisizioni_nave/nave_raw_data_c2.npy')
data2 = np.asarray(data2)
rdm2 = processing.processing_rangeDopplerData(data2)

In [27]:
data3 = np.load('acquisizioni_nave/nave_raw_data_c3.npy')
data3 = np.asarray(data3)
rdm3 = processing.processing_rangeDopplerData(data3)

#### Note: There is a function to get range_data called "processing_rangeData" which will be usefull to detect the movements.

In [ ]:
# The shape of the radar data depends on the radar settings.
# The different dimensions of the shape are (frame_index, receiver_index, chirp_index, sample_index).

print("(frame_index, receiver_index, chirp_index, sample_index)")
print(data.shape)

Noi acquisiamo 1000 frame per 3 canali, ogni frame è costituito da 64 chirps e 128 samples

In [ ]:
def insert(data,target):
    ant0=[]
    target1=[]
    for i in range(data.shape[0]): 
        ant0.append(data[i])
        
    if target ==0:
        target1 = [target]*1000
    elif target==1:
        target1 = [target]*1000
    elif target ==2:
        target1= [target]*3000
    elif target == 3:
        target1 = [target]*3000
        
    bello=pd.DataFrame({'Tot_signal':ant0,'target':target1})
    
    return bello

brutto = pd.concat([insert(rdm00,0),insert(rdm01,0),insert(rdm02,0),
                    insert(rdm10,1),insert(rdm11,1),insert(rdm12,1),
                    insert(rdm2,2),insert(rdm3,3)])


In [53]:
brutto

,Tot_signal,target
2995,"[[[(7.66053886991358e-15+0j), (-0.051498141870...",3
2996,"[[[(1.8041124150158794e-14+0j), (-0.0036276538...",3
2997,"[[[(1.887379141862766e-14+0j), (-0.01201945236...",3
2998,"[[[(-1.7541523789077473e-14+0j), (0.0356028726...",3
2999,"[[[(-2.8199664825478976e-14+0j), (-0.020159803...",3


In [54]:
brutto.to_csv('nave_map.csv',index=False)

In [ ]:
fig, axs = plot.subplots(3, 5,figsize=(15,10),sharex=True, sharey=  True)
fig.suptitle('Range-Doppler Plot')

for i in range(3):
    for j in range(5):
        axs[i, j].imshow(np.abs(range_doppler_map)[j,i,:,:])
        axs[i, j].set_aspect('equal')

plot.subplots_adjust(hspace=0)

In [ ]:
from keras import Model
from keras.layers import Dense

from keras_pandas.Automater import Automater
from keras_pandas.lib import load_titanic

observations = load_titanic()

# Transform the data set, using keras_pandas
numerical_vars = ['Tot_signal']


auto = Automater(categorical_vars=categorical_vars, numerical_vars=numerical_vars, text_vars=text_vars,
 response_var='survived')